In [5]:
# !pip install --upgrade mysql-connector-python
# !pip install sshtunnel
# !pip install pymysql
# !pip install pymysql

In [45]:
import pymysql
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

# DB 연결 설정 (본인의 DB 정보로 수정)
conn = pymysql.connect(
    host='localhost',
    user='dbuser',
    password='1234',
    db='project',
    charset='utf8'
)

# 데이터 가져오기
try:
    query = 'SELECT * FROM NAJU_WEATHER_TB'  # NAJU_WEATHER_TB 테이블에서 데이터 전체를 가져옴
    data = pd.read_sql(query, conn)
finally:
    conn.close()

# 시간 정보 처리
data['MEAS_DT'] = pd.to_datetime(data['MEAS_DT'])
data = data.sort_values('MEAS_DT')

# 내일 및 모레 타겟 생성
data['AP_tomorrow'] = data['AP'].shift(-24)
data['TEMP_tomorrow'] = data['TEMP'].shift(-24)
data['HUM_tomorrow'] = data['HUM'].shift(-24)
data['WS_tomorrow'] = data['WS'].shift(-24)
data['WD_tomorrow'] = data['WD'].shift(-24)
data['RF_tomorrow'] = data['RF'].shift(-24)

data['AP_day_after'] = data['AP'].shift(-48)
data['TEMP_day_after'] = data['TEMP'].shift(-48)
data['HUM_day_after'] = data['HUM'].shift(-48)
data['WS_day_after'] = data['WS'].shift(-48)
data['WD_day_after'] = data['WD'].shift(-48)
data['RF_day_after'] = data['RF'].shift(-48)

# NaN 값 제거
data = data.dropna()

# 특징 및 타겟 설정
features = ['AP', 'HUM', 'TEMP', 'WS', 'WD', 'RF']
targets_tomorrow = [
    'AP_tomorrow', 'TEMP_tomorrow', 'HUM_tomorrow', 
    'WS_tomorrow', 'WD_tomorrow', 'RF_tomorrow'
]
targets_day_after = [
    'AP_day_after', 'TEMP_day_after', 'HUM_day_after', 
    'WS_day_after', 'WD_day_after', 'RF_day_after'
]

X = data[features]
y_tomorrow = data[targets_tomorrow]
y_day_after = data[targets_day_after]

# 데이터 분리
X_train, X_test, y_train_tomorrow, y_test_tomorrow = train_test_split(X, y_tomorrow, test_size=0.2, random_state=42)
_, _, y_train_day_after, y_test_day_after = train_test_split(X, y_day_after, test_size=0.2, random_state=42)

# 모델 초기화 및 학습
model_tomorrow = RandomForestRegressor(random_state=42, n_estimators=100)
model_day_after = RandomForestRegressor(random_state=42, n_estimators=100)

model_tomorrow.fit(X_train, y_train_tomorrow)
model_day_after.fit(X_train, y_train_day_after)

# 예측
latest_data = X.iloc[-1:]  # 가장 최근 데이터를 기준으로 예측
predicted_tomorrow = model_tomorrow.predict(latest_data)
predicted_day_after = model_day_after.predict(latest_data)

# 예측 결과 출력
print("내일 예측:")
print(f"기압: {predicted_tomorrow[0][0]:.2f} hPa")
print(f"기온: {predicted_tomorrow[0][1]:.2f} °C")
print(f"습도: {predicted_tomorrow[0][2]:.2f} %")
print(f"풍속: {predicted_tomorrow[0][3]:.2f} m/s")
print(f"풍향: {predicted_tomorrow[0][4]:.2f} °")
print(f"강수: {predicted_tomorrow[0][5]:.2f} mm")

print("\n모레 예측:")
print(f"기압: {predicted_day_after[0][0]:.2f} hPa")
print(f"기온: {predicted_day_after[0][1]:.2f} °C")
print(f"습도: {predicted_day_after[0][2]:.2f} %")
print(f"풍속: {predicted_day_after[0][3]:.2f} m/s")
print(f"풍향: {predicted_day_after[0][4]:.2f} °")
print(f"강수: {predicted_day_after[0][5]:.2f} mm")


OperationalError: (2013, 'Lost connection to MySQL server during query ([WinError 10054] 현재 연결은 원격 호스트에 의해 강제로 끊겼습니다)')

In [47]:
# 필요한 라이브러리 불러오기
import pymysql
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score
from imblearn.over_sampling import SMOTE

# DB 연결 설정 (본인의 DB 정보로 수정)
conn = pymysql.connect(
    host='localhost',
    user='dbuser',
    password='1234',
    db='project',
    charset='utf8'
)

# 데이터 가져오기
try:
    query = 'SELECT * FROM NAJU_WEATHER_TB'  # 예시: NAJU_WEATHER_TB 테이블에서 데이터 전체를 가져옴
    df = pd.read_sql(query, conn)
finally:
    conn.close()



from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

# 데이터 불러오기
file_path = df  # 파일 경로를 실제 파일 경로로 변경
data = pd.read_csv(file_path)

# 시간 정보 처리
data['MEAS_DT'] = pd.to_datetime(data['MEAS_DT'])
data = data.sort_values('MEAS_DT')

# 내일 및 모레 타겟 생성
data['AP_tomorrow'] = data['AP'].shift(-24)
data['TEMP_tomorrow'] = data['TEMP'].shift(-24)
data['HUM_tomorrow'] = data['HUM'].shift(-24)
data['WS_tomorrow'] = data['WS'].shift(-24)
data['WD_tomorrow'] = data['WD'].shift(-24)
data['RF_tomorrow'] = data['RF'].shift(-24)

data['AP_day_after'] = data['AP'].shift(-48)
data['TEMP_day_after'] = data['TEMP'].shift(-48)
data['HUM_day_after'] = data['HUM'].shift(-48)
data['WS_day_after'] = data['WS'].shift(-48)
data['WD_day_after'] = data['WD'].shift(-48)
data['RF_day_after'] = data['RF'].shift(-48)

# NaN 값 제거
data = data.dropna()

# 특징 및 타겟 설정
features = ['AP', 'HUM', 'TEMP', 'WS', 'WD', 'RF']
targets_tomorrow = [
    'AP_tomorrow', 'TEMP_tomorrow', 'HUM_tomorrow', 
    'WS_tomorrow', 'WD_tomorrow', 'RF_tomorrow'
]
targets_day_after = [
    'AP_day_after', 'TEMP_day_after', 'HUM_day_after', 
    'WS_day_after', 'WD_day_after', 'RF_day_after'
]

X = data[features]
y_tomorrow = data[targets_tomorrow]
y_day_after = data[targets_day_after]

# 데이터 분리
X_train, X_test, y_train_tomorrow, y_test_tomorrow = train_test_split(X, y_tomorrow, test_size=0.2, random_state=42)
_, _, y_train_day_after, y_test_day_after = train_test_split(X, y_day_after, test_size=0.2, random_state=42)

# 모델 초기화 및 학습
model_tomorrow = RandomForestRegressor(random_state=42, n_estimators=100)
model_day_after = RandomForestRegressor(random_state=42, n_estimators=100)

model_tomorrow.fit(X_train, y_train_tomorrow)
model_day_after.fit(X_train, y_train_day_after)

# 예측
latest_data = X.iloc[-1:]  # 가장 최근 데이터를 기준으로 예측
predicted_tomorrow = model_tomorrow.predict(latest_data)
predicted_day_after = model_day_after.predict(latest_data)

# 예측 결과 출력
print("내일 예측:")
print(f"기압: {predicted_tomorrow[0][0]:.2f} hPa")
print(f"기온: {predicted_tomorrow[0][1]:.2f} °C")
print(f"습도: {predicted_tomorrow[0][2]:.2f} %")
print(f"풍속: {predicted_tomorrow[0][3]:.2f} m/s")
print(f"풍향: {predicted_tomorrow[0][4]:.2f} °")
print(f"강수: {predicted_tomorrow[0][5]:.2f} mm")

print("\n모레 예측:")
print(f"기압: {predicted_day_after[0][0]:.2f} hPa")
print(f"기온: {predicted_day_after[0][1]:.2f} °C")
print(f"습도: {predicted_day_after[0][2]:.2f} %")
print(f"풍속: {predicted_day_after[0][3]:.2f} m/s")
print(f"풍향: {predicted_day_after[0][4]:.2f} °")
print(f"강수: {predicted_day_after[0][5]:.2f} mm")

# # 데이터 전처리 및 모델링
# # RF 열을 숫자형으로 변환 (float으로 변환하고 오류 발생 시 NaN 처리)
# df['RF'] = pd.to_numeric(df['RF'], errors='coerce').fillna(0)

# # 비 데이터(RF)를 기준으로 이진화 (비가 내렸는지 여부)
# df['Rain'] = df['RF'].apply(lambda x: 1 if x > 0 else 0)

# # 특성과 레이블 선택
# X = df[['AP', 'HUM', 'TEMP', 'WS', 'WD']]  # 예측 변수 선택
# y = df['Rain']  # 타깃 변수

# # 데이터 분할
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# # 데이터 불균형 해결을 위해 SMOTE 적용
# smote = SMOTE(random_state=42)
# X_train_res, y_train_res = smote.fit_resample(X_train, y_train)

# # 모델 학습
# model = RandomForestClassifier(random_state=42)
# model.fit(X_train_res, y_train_res)

# # 예측
# y_pred = model.predict(X_test)

# # 예측 결과와 실제값을 포함한 결과 DataFrame 생성
# results_df = X_test.copy()
# results_df['Actual'] = y_test.values
# results_df['Predicted'] = y_pred

# # 성능 평가
# print(classification_report(y_test, y_pred))
# print("Accuracy:", accuracy_score(y_test, y_pred))

# # 결과 DataFrame 출력 (원하는 경우 CSV로 저장)
# print(results_df.head())  # 결과 미리보기
# # results_df.to_csv("rain_prediction_results.csv", index=False)  # CSV 파일로 저장 가능


OperationalError: (2003, "Can't connect to MySQL server on 'localhost' ([WinError 10061] 대상 컴퓨터에서 연결을 거부했으므로 연결하지 못했습니다)")

결과에 대한 의미를 간단히 설명드리면 다음과 같습니다.

성능 평가 지표
Precision (정밀도): 모델이 '비가 올 것'이라고 예측한 경우 중 실제로 비가 내린 경우의 비율입니다.

0 (비가 안 옴): 99%
1 (비가 옴): 52%

Recall (재현율): 실제 비가 내린 경우 중에서 모델이 '비가 올 것'이라고 올바르게 예측한 비율입니다.

0 (비가 안 옴): 98%
1 (비가 옴): 71%
F1-Score: Precision과 Recall의 조화 평균으로, 두 지표를 모두 고려한 성능 척도입니다.

0 (비가 안 옴): 0.99
1 (비가 옴): 0.60
Accuracy (정확도): 전체 데이터 중 모델이 정확하게 예측한 비율로 97.33%의 높은 정확도를 보입니다. 하지만 클래스 1 (비가 옴)의 경우 예측 성능이 낮아 상대적으로 성능이 떨어집니다.

데이터의 클래스 불균형
0 (비가 안 옴): 583건
1 (비가 옴): 17건
비가 내리는 경우가 드물어서 모델이 비가 내리지 않는 경우 (0)를 매우 잘 예측하지만, 비가 오는 경우 (1)는 상대적으로 잘 예측하지 못하는 편입니다.

예측 결과 예시
아래의 표는 각 샘플의 예측 값과 실제 값을 보여줍니다.

AP	HUM	TEMP	WS	WD	Actual	Predicted
1029.9	51	0.3	3.3	13.7	0	0
1024.1	93	-4.1	0.7	351.3	0	0
1026.9	88	-8	0.2	0	0	0
1013.3	76	10	0.8	104.1	0	0
1031.5	85	-7	1.9	40.3	0	0
Actual: 실제로 비가 내린 여부 (1이면 비가 옴, 0이면 비가 안 옴)
Predicted: 모델이 예측한 값

In [9]:
filtered_df = df.drop(columns=['MEAS_DT', 'REG_ID', 'REG_DT'])
print(filtered_df)



       DATA_ID      AP  HUM  TEMP   WS     WD   RF  Rain
0            1  1024.3   99   8.5  0.3      0  0.0     0
1            2  1024.3   99   8.3  0.5   11.8  0.0     0
2            3  1023.8  100   8.6  0.7   48.7  0.0     0
3            4  1023.7  100   9.3  1.7   46.8  0.0     0
4            5  1023.4  100   8.7  1.1   79.8  0.0     0
...        ...     ...  ...   ...  ...    ...  ...   ...
25470    26280  1021.5   95  13.6    0      0  0.0     0
25471    26281    1022   94    14  1.6    157  0.0     0
25472    26282    1022   95  12.9  1.2  164.3  0.0     0
25473    26283  1021.9   98  11.9  0.7  174.9  0.0     0
25474    26284  1022.1   98  10.1    0      0  0.0     0

[25475 rows x 8 columns]


In [11]:
print(filtered_df['Rain'].value_counts())

Rain
0    23963
1     1512
Name: count, dtype: int64


In [13]:
# Rain이 0인 데이터
print(filtered_df[filtered_df['Rain'] == 0])

# Rain이 1인 데이터
print(filtered_df[filtered_df['Rain'] == 1])

       DATA_ID      AP  HUM  TEMP   WS     WD   RF  Rain
0            1  1024.3   99   8.5  0.3      0  0.0     0
1            2  1024.3   99   8.3  0.5   11.8  0.0     0
2            3  1023.8  100   8.6  0.7   48.7  0.0     0
3            4  1023.7  100   9.3  1.7   46.8  0.0     0
4            5  1023.4  100   8.7  1.1   79.8  0.0     0
...        ...     ...  ...   ...  ...    ...  ...   ...
25470    26280  1021.5   95  13.6    0      0  0.0     0
25471    26281    1022   94    14  1.6    157  0.0     0
25472    26282    1022   95  12.9  1.2  164.3  0.0     0
25473    26283  1021.9   98  11.9  0.7  174.9  0.0     0
25474    26284  1022.1   98  10.1    0      0  0.0     0

[23963 rows x 8 columns]
       DATA_ID      AP HUM  TEMP   WS     WD   RF  Rain
175        176  1011.1  93  14.3  1.9  274.8  5.5     1
176        177  1012.7  92  12.7  3.6  283.4  4.5     1
177        178  1014.5  92  11.4  3.3  285.9  3.5     1
178        179  1015.4  90  10.4  4.6  283.6  2.5     1
179       

In [5]:
# 데이터 전처리 및 모델링
# RF 열을 숫자형으로 변환 (float으로 변환하고 오류 발생 시 NaN 처리)
df['RF'] = pd.to_numeric(df['RF'], errors='coerce').fillna(0)

# 비 데이터(RF)를 기준으로 이진화 (비가 내렸는지 여부)
df['Rain'] = df['RF'].apply(lambda x: 1 if x > 0 else 0)

# 특성과 레이블 선택
X = df[['AP', 'HUM', 'TEMP', 'WS', 'WD']]  # 예측 변수 선택
y = df['Rain']  # 타깃 변수

# 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 데이터 불균형 해결을 위해 SMOTE 적용
smote = SMOTE(random_state=42)
X_train_res, y_train_res = smote.fit_resample(X_train, y_train)

# 모델 학습
model = RandomForestClassifier(random_state=42)
model.fit(X_train_res, y_train_res)

# 예측
y_pred = model.predict(X_test)

# 예측 결과와 실제값을 포함한 결과 DataFrame 생성
results_df = X_test.copy()
results_df['Actual'] = y_test.values
results_df['Predicted'] = y_pred

# 성능 평가
print(classification_report(y_test, y_pred))
print("Accuracy:", accuracy_score(y_test, y_pred))

# 결과 DataFrame 출력 (원하는 경우 CSV로 저장)
print(results_df.head())  # 결과 미리보기
# results_df.to_csv("rain_prediction_results.csv", index=False)  # CSV 파일로 저장 가능

              precision    recall  f1-score   support

           0       0.97      0.96      0.96      4792
           1       0.45      0.49      0.47       303

    accuracy                           0.93      5095
   macro avg       0.71      0.73      0.72      5095
weighted avg       0.94      0.93      0.94      5095

Accuracy: 0.9338567222767419
           AP HUM  TEMP   WS     WD  Actual  Predicted
7745   1014.1  70    23  1.6   41.2       0          0
13523  1010.5  98  20.8  0.1      0       0          0
16670  1023.9  88  14.1  0.5  285.4       0          0
18340  1027.9  77   0.4  4.8  329.8       0          0
2734   1024.1  67   0.7  1.7   53.2       0          0


In [17]:
import pandas as pd

# 사용자로부터 과거 특정 날짜의 날씨 데이터 입력
ap = float(input("기압 (AP): "))
hum = float(input("습도 (HUM): "))
temp = float(input("온도 (TEMP): "))
ws = float(input("풍속 (WS): "))
wd = float(input("풍향 (WD): "))

# 입력받은 값을 데이터프레임으로 변환
past_day_data = {
    'AP': [ap],
    'HUM': [hum],
    'TEMP': [temp],
    'WS': [ws],
    'WD': [wd]
}
past_day_df = pd.DataFrame(past_day_data)

# 예측 수행
predicted_rain = model.predict(past_day_df)

# 예측 결과 출력
if predicted_rain[0] == 1:
    print("예측 결과: 비가 오는 날입니다.")
else:
    print("예측 결과: 비가 오지 않는 날입니다.")


기압 (AP):  100
습도 (HUM):  100
온도 (TEMP):  100
풍속 (WS):  100
풍향 (WD):  10


예측 결과: 비가 오는 날입니다.


In [149]:
import pandas as pd

class WeatherPredictor:
    def __init__(self, model):
        # 모델을 초기화합니다.
        self.model = model

    def get_input_data(self):
        # 사용자 입력을 받아 데이터프레임으로 변환합니다.
        ap = float(input("기압 (AP): "))
        hum = float(input("습도 (HUM): "))
        temp = float(input("온도 (TEMP): "))
        ws = float(input("풍속 (WS): "))
        wd = float(input("풍향 (WD): "))
        
        past_day_data = {
            'AP': [ap],
            'HUM': [hum],
            'TEMP': [temp],
            'WS': [ws],
            'WD': [wd]
        }
        return pd.DataFrame(past_day_data)

    def predict_rain(self, input_data):
        # 입력 데이터를 기반으로 비가 오는지 예측합니다.
        predicted_rain = self.model.predict(input_data)
        
        # 예측 결과를 반환합니다.
        return predicted_rain[0]

    def display_prediction(self, prediction):
        # 예측 결과를 출력합니다.
        if prediction == 1:
            print("예측 결과: 비가 오는 날입니다.")
        else:
            print("예측 결과: 비가 오지 않는 날입니다.")

# 사용 예시
# model = ... # 사전에 훈련된 모델을 불러와야 합니다.
weather_predictor = WeatherPredictor(model)

# 데이터 입력
input_data = weather_predictor.get_input_data()

# 예측 수행
prediction = weather_predictor.predict_rain(input_data)

# 결과 출력
weather_predictor.display_prediction(prediction)


기압 (AP):  1000
습도 (HUM):  60
온도 (TEMP):  20
풍속 (WS):  3.4
풍향 (WD):  100


예측 결과: 비가 오지 않는 날입니다.


In [19]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import GridSearchCV

# 데이터 불균형 해결을 위해 SMOTE 적용
smote = SMOTE(random_state=42)
X_train_res, y_train_res = smote.fit_resample(X_train, y_train)

# RandomForest 모델에 class_weight 옵션 적용
rf_model = RandomForestClassifier(random_state=42, class_weight='balanced')
rf_model.fit(X_train_res, y_train_res)

# 예측
y_pred_rf = rf_model.predict(X_test)

# 성능 평가
print("Random Forest with Balanced Class Weight")
print(classification_report(y_test, y_pred_rf))
print("Accuracy:", accuracy_score(y_test, y_pred_rf))


Random Forest with Balanced Class Weight
              precision    recall  f1-score   support

           0       0.97      0.96      0.96      4792
           1       0.45      0.49      0.47       303

    accuracy                           0.93      5095
   macro avg       0.71      0.73      0.72      5095
weighted avg       0.94      0.93      0.94      5095

Accuracy: 0.9338567222767419


In [21]:
# 하이퍼파라미터 튜닝을 위한 파라미터 그리드 정의
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [10, 20, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
}

# GridSearchCV로 최적의 하이퍼파라미터 찾기
grid_search = GridSearchCV(RandomForestClassifier(random_state=42, class_weight='balanced'), 
                           param_grid, cv=3, scoring='f1', n_jobs=-1)
grid_search.fit(X_train_res, y_train_res)

# 최적의 모델로 예측 수행
best_rf_model = grid_search.best_estimator_
y_pred_best_rf = best_rf_model.predict(X_test)

# 성능 평가
print("Best Random Forest Model after Hyperparameter Tuning")
print(classification_report(y_test, y_pred_best_rf))
print("Accuracy:", accuracy_score(y_test, y_pred_best_rf))


KeyboardInterrupt: 

In [25]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.preprocessing import MinMaxScaler

# 데이터 스케일링
scaler = MinMaxScaler()
temp_scaled = scaler.fit_transform(df[['TEMP']].values)

# 시계열 데이터 준비 (LSTM 입력에 맞게 변환)
def create_sequences(data, seq_length):
    X, y = [], []
    for i in range(len(data) - seq_length):
        X.append(data[i:i + seq_length])
        y.append(data[i + seq_length])
    return np.array(X), np.array(y)

seq_length = 30  # 최근 30일의 데이터를 사용
X, y = create_sequences(temp_scaled, seq_length)

# 훈련 및 테스트 분할
X_train, X_test = X[:-1], X[-1:]
y_train, y_test = y[:-1], y[-1:]

# LSTM 모델 생성
model = Sequential([
    LSTM(50, activation='relu', input_shape=(X_train.shape[1], X_train.shape[2])),
    Dense(1)
])
model.compile(optimizer='adam', loss='mse')
model.fit(X_train, y_train, epochs=50, batch_size=16, verbose=1)

# 내일 기온 예측
tomorrow_temperature_scaled = model.predict(X_test)
tomorrow_temperature = scaler.inverse_transform(tomorrow_temperature_scaled)
print("내일 예상 온도:", tomorrow_temperature[0][0])


Epoch 1/50
1591/1591 [==============================] - 10s 6ms/step - loss: 0.0047
Epoch 2/50
1591/1591 [==============================] - 9s 5ms/step - loss: 3.4696e-04
Epoch 3/50
1591/1591 [==============================] - 9s 5ms/step - loss: 2.9646e-04
Epoch 4/50
1591/1591 [==============================] - 9s 5ms/step - loss: 2.8505e-04
Epoch 5/50
1591/1591 [==============================] - 9s 5ms/step - loss: 2.7724e-04
Epoch 6/50
1591/1591 [==============================] - 9s 5ms/step - loss: 2.7415e-04
Epoch 7/50
1591/1591 [==============================] - 9s 6ms/step - loss: 2.6536e-04
Epoch 8/50
1591/1591 [==============================] - 9s 6ms/step - loss: 2.7713e-04
Epoch 9/50
1591/1591 [==============================] - 9s 6ms/step - loss: 2.6359e-04
Epoch 10/50
1591/1591 [==============================] - 9s 6ms/step - loss: 2.6270e-04
Epoch 11/50
1591/1591 [==============================] - 9s 6ms/step - loss: 2.5835e-04
Epoch 12/50
1591/1591 [=====================

In [29]:
# 모델 저장
model.save("temperature_prediction_model.h5")


In [31]:
from tensorflow.keras.models import load_model

# 모델 불러오기
loaded_model = load_model("temperature_prediction_model.h5")

# 내일 기온 예측 (불러온 모델 사용)
tomorrow_temperature_scaled = loaded_model.predict(X_test)
tomorrow_temperature = scaler.inverse_transform(tomorrow_temperature_scaled)
print("내일 예상 온도 (불러온 모델):", tomorrow_temperature[0][0])


UnicodeDecodeError: 'utf-8' codec can't decode byte 0xb0 in position 43: invalid start byte